[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeteoSwiss/opendata-nwp-demos/blob/main/06_calculate_global_rad_flux.ipynb)

# Calculate Global Radiation Flux from the Direct Model Outputs of ICON-CH1-EPS

This notebook demonstrates the full workflow for accessing ICON-CH1-EPS forecasts and computing the **global radiation flux** `GLOB`. The data is provided by MeteoSwiss as part of Switzerland’s [Open Government Data (OGD) initiative](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html).

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed in-house to simplify working with numerical weather model data. It includes the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module for data access, along with utilities for applying temporal and spatial operators to model output.

Many users are particularly interested in the **Global Radiation Flux** `GLOB`, which is not available as a direct model output. Instead, it must be **derived** using the downward direct and diffuse short wave radiation fluxes at the surface (`ASWDIR_S` and `ASWDIFD_S`):

$$GLOB = ASWDIR\_S + ASWDIFD\_S$$
Note that the original data is **aggregated since the initialization time of the forecast**. Therefore, if the desired output is an hourly average of the flux, a de-aggregation step is required.

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch deterministic ICON-CH1-EPS forecast data (e.g., downward direct and diffuse short wave radiation fluxes at the surface `ASWDIR_S` and `ASWDIFD_S`) using [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/)'s [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module.

 ⚙️  **De-aggregate to obtain hourly average flux**  
    The flux data has been aggregated since the forecast initialization. To get hourly averages, we use the [resample_average](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.time_operators.html#meteodatalab.operators.time_operators.resample_average) time operator from [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).

 📈  **Compute Global Radiation Flux**  
    Compute the Global Radiation Flux by combining the direct and diffuse shortwave radiation fluxes at the surface using the [compute_swdown](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.radiation.html#meteodatalab.operators.radiation.compute_swdown) operator from [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).

---

## Retrieving Forecasts
In this first part, we retrieve deterministic ICON-CH1-EPS downward direct and diffuse short wave radiation fluxes forecasts. To access this data, we use the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON-CH1/2-EPS forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2.png](./images/browser-ch2.png)

> ⚙️ **Google Colab Setup**  
> This cell installs all required dependencies if you're running the notebook in **Google Colab**.  
> It is automatically skipped when running in a local Jupyter environment.

In [1]:
# 📦 Google Colab Setup (skipped if not running in Colab)
import sys

def is_colab():
    return "google.colab" in sys.modules

if is_colab():
    !git clone https://github.com/MeteoSwiss/opendata-nwp-demos.git
    %cd opendata-nwp-demos
    !pip install poetry && poetry config virtualenvs.in-project true && poetry install --no-ansi
    import sys, os, pathlib
    venv = pathlib.Path(".venv")
    site = venv / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"
    sys.path.insert(0, str(site))
    os.environ["ECCODES_DEFINITION_PATH"] = str((venv / "share/eccodes-cosmo-resources/definitions").resolve())

### Creating Requests
To retrieve the forecast data, we first define API requests using the [ogd_api.Request](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.Request) class. In this example, we create two requests: one to retrieve the direct short wave radiation flux (`ASWDIR_S`) and the diffuse short wave radiation flux (`ASWDIFD_S`) to obtain the global radiation flux `GLOB`.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

>ⓘ **Number of steps retrieved**: ICON-CH1-EPS provides 34 hourly steps (from hour 0 to +33), which typically takes around 3 minutes to retrieve. For the sake of this example, we limit the retrieval to 10 steps to save time.

In [2]:
from datetime import datetime, timedelta, timezone
from meteodatalab import ogd_api

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

param_list = ["ASWDIR_S", "ASWDIFD_S"]
req_list = []

for param in param_list:
    req = ogd_api.Request(
        collection="ogd-forecasting-icon-ch1",
        variable=param,
        ref_time=today_midnight_utc,
        perturbed=False,
        lead_time=[timedelta(hours=h) for h in range(10)], # Retrieve the first 10 hourly steps of ICON-CH1-EPS (from hour 0 to +9)
    )
    req_list.append((param,req))


Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch1` for ICON-CH1-EPS). |
| `variable`           | Meteorological variable of interest (`ASWDIR_S` = direct short wave radiation flux and `ASWDIFD_S` = diffuse short wave radiation flux). |
| `ref_time` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `lead_time`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=0)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT0H"`) <br>– A list of `datetime.timedelta` objects or of `ISO 8601` duration strings (e.g., `[datetime.timedelta(hours=h) for h in range(34)]` |

### Retrieving Data
We now send our list of requests to the API and retrieve the resulting datasets using the [get_from_ogd()](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.get_from_ogd) function.
Each response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing `HTTPError: 403 Client Error: Forbidden for url`, you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [3]:
from earthkit.data import config
config.set("cache-policy", "temporary")

da_dict = {}
for param, req in req_list:
    da = ogd_api.get_from_ogd(req)
    da_dict[param]= da

The result of each `ogd_api.get_from_ogd()` call is an `xarray.DataArray` with the following structure:

**Dimensions**
- `eps` (ensemble members): 1 member (for deterministic data)
- `ref_time`: forecast init time (UTC)
- `lead_time`: time offset from `ref_time`
- `z`: vertical model levels (80)
- `cell`: spatial grid points (1'147'980 for ICON-CH1-EPS)

**Coordinates**
- `lon` / `lat`: longitude & latitude per `cell`
- `valid_time`: `ref_time + lead_time`
        
**Attributes**
- `parameter`: variable info (e.g., name, units)
- `vcoord_type`: vertical coordinate type (e.g., `surface`)
- `geography`: grid structure metadata

In [4]:
da_dict["ASWDIR_S"]

<xarray.DataArray (eps: 1, ref_time: 1, lead_time: 10, cell: 1147980)> Size: 46MB
array([[[[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ],
         [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ],
         [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ],
         ...,
         [ 0.      ,  0.      ,  0.      , ..., 63.75    , 63.177734,
          64.0293  ],
         [ 0.      ,  0.      ,  0.      , ..., 76.31641 , 74.86719 ,
          73.97266 ],
         [ 0.      ,  0.      ,  0.      , ..., 85.84766 , 84.08984 ,
          80.89453 ]]]], dtype=float32)
Coordinates:
  * eps         (eps) int64 8B 0
  * ref_time    (ref_time) datetime64[ns] 8B 2025-06-05
  * lead_time   (lead_time) timedelta64[ns] 80B 00:00:00 01:00:00 ... 09:00:00
    lon         (cell) float32 5MB 17.71 17.69 17.7 17.7 ... 12.81 12.81 12.8
    lat         (cell) float32 5MB 50.24 50.24 50.24 50.23 ... 50.06 50.05 50.04
    valid_time  (ref_time, lead_time) datetime64[ns] 80B 2025-06-05 ... 2025-...
Dimensions without coordinates: cell
Attributes:
    metadata:     <earthkit.data.readers.grib.metadata.StandAloneGribMetadata...
    parameter:    {'centre': 'lssw', 'paramId': 500480, 'shortName': 'ASWDIR_...
    geography:    {'gridType': 'unstructured_grid', 'bitmapPresent': 0}
    vref:         geo
    vcoord_type:  surface
    origin_z:     0.0

> &#x1F525; **Expert users:** If you are interested in detailed metadata information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

### De-aggregate to obtain hourly average flux  
The flux data has been aggregated from the forecast initialization time, meaning each value represents an average from hour 0 up to the given lead time. This aggregation behavior is defined in the metadata of the original files, known as GRIB keys.

You can find this information in the list of parameters available for each collection. For each parameter, details such as the unit, level type, and temporal aggregation method are provided.  
- 🔗 You can download the CSV file from the [ICON-CH1-EPS collection page](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en&.asset=asset-params.csv), using the section highlighted in the image below:




<img src="./images/param_overview_browser.png" alt="param_overview_browser" width="600"/>



For example, here’s a sample row for the parameter `ASWDIFD_S`:
| Parameter   | Longname                                      | Standard Unit | Level         | Vertical Coordinate       | Horizon | **Temporal Aggregation**              | **Start of Temporal Aggregation** |
|-------------|-----------------------------------------------|----------------|----------------|----------------------------|---------|------------------------------------|-------------------------------|
| ASWDIFD_S   | Downward diffuse short wave radiation flux at surface | W m⁻²        | Single Level   | Surface (no dimension)     | All     | **Average (mean over forecast time)** | **Reference Time**                |

In this case, the parameter is provided as a mean over the entire forecast time. For `ASWDIR_S`, the aggregation method is even indicated in the parameter name itself:

In [5]:
da_dict["ASWDIR_S"].attrs["parameter"]["name"] 

'Downward direct short wave radiation flux at surface (mean over forecast time)'



However, for this example, we want to obtain **hourly average values** instead. To do this, we apply the [resample_average](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.time_operators.html#meteodatalab.operators.time_operators.resample_average) time operator from [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/). This operator computes averages with respect to the given time interval for every lead time present in the input field.
In our case, the interval is 1 hour, specified as `np.timedelta64(1, "h")`.

In [6]:
import meteodatalab.operators.time_operators as time_ops
import numpy as np

direct_rad_resampled = time_ops.resample_average(da_dict["ASWDIR_S"], np.timedelta64(1, "h"))
diffuse_rad_resampled = time_ops.resample_average(da_dict["ASWDIFD_S"], np.timedelta64(1, "h"))

### Compute Global Radiation Flux  

We compute `GLOB` using the resampled `ASWDIR_S` and `ASWDIFD_S` components, which share the same dimensions. To perform the calculation, we use the [compute_swdown](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.radiation.html#meteodatalab.operators.radiation.compute_swdown) function from [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).
The result is a `xarray.DataArray` with the same dimensions.

In [7]:
from meteodatalab.operators import radiation

global_rad = radiation.compute_swdown(diffuse_rad_resampled, direct_rad_resampled)
global_rad

<xarray.DataArray (eps: 1, ref_time: 1, lead_time: 10, cell: 1147980)> Size: 92MB
array([[[[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         ...,
         [  0.        ,   0.        ,   0.        , ..., 399.31835938,
          396.84570312, 395.09765625],
         [  0.        ,   0.        ,   0.        , ..., 454.12890625,
          447.37695312, 432.9921875 ],
         [  0.        ,   0.        ,   0.        , ..., 482.9296875 ,
          473.83203125, 444.56640625]]]])
Coordinates:
  * eps         (eps) int64 8B 0
  * ref_time    (ref_time) datetime64[ns] 8B 2025-06-05
  * lead_time   (lead_time) timedelta64[ns] 80B 00:00:00 01:00:00 ... 09:00:00
    lon         (cell) float32 5MB 17.71 17.69 17.7 17.7 ... 12.81 12.81 12.8
    lat         (cell) float32 5MB 50.24 50.24 50.24 50.23 ... 50.06 50.05 50.04
    valid_time  (ref_time, lead_time) datetime64[ns] 80B 2025-06-05 ... 2025-...
Dimensions without coordinates: cell
Attributes:
    metadata:     <earthkit.data.readers.grib.metadata.StandAloneGribMetadata...
    parameter:    {'centre': 'lssw', 'paramId': 503174, 'shortName': 'ASOD_S'...
    geography:    {'gridType': 'unstructured_grid', 'bitmapPresent': 0}
    vref:         geo
    vcoord_type:  surface
    origin_z:     0.0

As we print the output array, we can also observe that its metadata has been updated.
The new parameter name is stored in:

In [8]:
global_rad.attrs["parameter"]["name"]

'Downward short wave radiation flux at surface (time average)'